Chatgpt is a probabilistic system and for any one prompt it can give multiple answers sort of replying to it.
It is called language models because it models the sequence of words or characters or tokens and it know how sort of words follow each other in English language. It is completing the sequence: I give it the start of sequence and it completes the sequence with outcome.

Under the hood components
GPT(Generatively Pre-trained Transformer) Transformer: Neural Network  (does all the heavy lifting)

https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing

# Building a GPT
Companion notebook to the Zero To Hero video on GPT.

In [6]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-16 04:40:59--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1,06M  2,33MB/s    in 0,5s    

2024-04-16 04:41:00 (2,33 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [7]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [8]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [9]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [10]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


We would like to develop some strategy to 'tokenize' the input text. When people say tokenize, they mean convert the raw text as a string to some sequence of integers acoording to some vocabulary of possible elements. Here, we're going to be building a character level language model, so we're simply going to be translating individual characters into integers. 

We're building both the encoder and the decoder

In [12]:
#create a mapping from chracters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) #encoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


this is one way of tokenization. 
Google uses 'sentencepiece', it encodes text into integers(but in a different schema and using different vocabulary), it is a sub-word sort of tokenizer. (you're not encoding entire words, but you're not also encoding individual characters)

OpenAI has library called 'tiktoken', it uses Byte-Pair Encoding (BPE) tokenizer.
https://huggingface.co/learn/nlp-course/en/chapter6/5
getting the encoding for gpt2, instaed of having just 65 possible characters or tokens, they have 50K tokens.when they encode the exasct same "hii there", we only get a list of three integers.  

You can trade off the code book size and the sequence lengths. So you can have a very long sequences of integers with very small vocabularies, or you can have short sequences of inetgers with very large vocabularies. 
Typically, people use in practice the sub-word encoding. We will use character level tokenizer for the simplicity. we have very snall code books, we have very simple encode and decode functions, but we do get ery long sequences as a result.


In [14]:
import tiktoken
enc= tiktoken.get_encoding('gpt2')
print(enc.n_vocab)
print(enc.encode("hii there"))
print(enc.decode([71, 4178, 612]))

50257
[71, 4178, 612]
hii there


In [15]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [16]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

It will help us to understand, to what extent our model is overfitting. We're going to basically hide and keep the validation data on the side because we don't want just a perfect memorization of this exact Shakespeare.

## Data Loader: Batches of Chunks of Data

Note:
We're never going to feed the entire text to Transformer all at once. That would be computationally very expensive and prohibitive. 
We only work with the chunks of the dataset and when we train the transformer, we basically sample random chunks ot of the training set and train them just chunks at a time. These chunks have some kind of a length(maximum length)


In [17]:
block_size = 8
train_data[:block_size +1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

This has multiple examples packed into it that's beacause all of these characters follow each other. We're going to simultaneously train it to make predictions at every one of these positions. 
In a chunk of nine characters, there's actually actually 8 individual examples packed in there. 
Example: in the context of 18 -> 47 likely comes next, in the context of 18 47 -> 56 likelt comes next, in the context of 16 47 56 -> 58 likely comes next

In [19]:
x = train_data[:block_size] # inputs to the Transformer
y = train_data[1:block_size+1] # next block size characters(offset by one) y are the targets for each position in the input
for t in range(block_size):
    context = x[:t+1]
    target=y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


We train on all the 8 examples here with context one all the way up to context of block size. 
We train on that not just for computational reasons, it's also done to make the transformer network be used to seeing contexts all the way from one to block size <br>
We'd like to the transform to be used to seeing everthing in between and that's going to be useful during inference. Because while we're sampling, we can start the sampling generation with as little as one chracter of context and the Transformer knows how to predict the next character with all the way up to just one context of one and so then it can predict everthing up to block size. after block size we have to strat truncating because the Transformer will never receive more than block size inputs when predicting the next character. 


There is one more dimension to care about: **batch dimension**. 
As we're sampling these chunks of text, every time we're going to feed them into Transformer, we're going to have many batches of multiple chunks of text that are stacked up in a single tensor. That's just done for efficiency, so that we can keep the gpus busy because they're really good at parallel processing of data. and we want to process multiple chunks all at the sime time but those chunks are processed completely independently, they don't talk to each other. 

In [27]:
torch.manual_seed(1337)
batch_size = 4 # how many independet sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix]) #takes all one dimensional tensors and stacks them up at row, they all become a row in a 4x8 tensor
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("----")

for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")
    


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

this 4 * 8 array contains a total of 32 examples and they are completely independent as far as the Transformer concerned. 

These are 32 independent examples packed in to a single batch of the input X and then he desired targets are in y. this integer tensor of X is going to feed into transformer and that transformer is going to simultaneously process all these examples and then look up the correct integers to predict every one of these positions in the tensor y.


# Simplest Baseline : Bigram Language Model, Loss, Generation

In [29]:
print(xb) # our input to the transformer 

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [30]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        #idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) C:channel which is vocab_size #logits the scores for the next character in  the sequence
        return logits #we're predicting what comes next based on just the individual identity of a single token


m=BigramLanguageModel(vocab_size)
out= m(xb, yb)
print(out.shape)

torch.Size([4, 8, 65])
